In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [14]:
spark = SparkSession.builder.appName("EarthquakeData").getOrCreate()

# Read CSV into PySpark DataFrame
df = spark.read.csv("/Users/pruth/Downloads/data/database.csv", header=True, inferSchema=True)

In [ ]:
# Write DataFrame to MySQL
df.write.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/earthquake_db",
    driver="com.mysql.jdbc.Driver",
    dbtable="neic_earthquakes",
    user="root",
    password="root"
).mode("overwrite").save()

In [ ]:
# Read data from MySQL
df_mysql = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/earthquake_db",
    driver="com.mysql.jdbc.Driver",
    dbtable="neic_earthquakes",
    user="root",
    password="root"
).load()

In [ ]:
#1.How does the Day of a Week affect the number of earthquakes?
day_of_week_counts = df_mysql.groupBy(dayofweek("Date").alias("DayOfWeek")).count()
day_of_week_counts.show()

In [ ]:
#2.What is the relation between Day of the month and Number of earthquakes that happened in a year?
day_month_year_counts = df_mysql.groupBy(dayofmonth("Date").alias("DayOfMonth"), year("Date").alias("Year")).count()
day_month_year_counts.show()

In [ ]:
#3.What does the average frequency of earthquakes in a month from the year 1965 to 2016 tell us?
avg_frequency_by_month = df_mysql.filter((col("Year") >= 1965) & (col("Year") <= 2016)).groupBy(month("Date").alias("Month")).agg(avg("count").alias("AverageFrequency"))
avg_frequency_by_month.show()

In [ ]:
#4.What is the relation between Year and Number of earthquakes that happened in that year?
yearly_counts = df_mysql.groupBy("Year").count()
yearly_counts.show()

In [ ]:
#5.How has the earthquake magnitude on average been varied over the years?
avg_magnitude_by_year = df_mysql.groupBy("Year").agg(avg("Magnitude").alias("AverageMagnitude"))
avg_magnitude_by_year.show()

In [ ]:
#6.How does year impact the standard deviation of the earthquakes?
stddev_by_year = df_mysql.groupBy("Year").agg(stddev("Magnitude").alias("MagnitudeStdDev"))
stddev_by_year.show()


In [ ]:
#7.Does geographic location have anything to do with earthquakes?
location_counts = df_mysql.groupBy("Location").count()
location_counts.show()

In [ ]:
#8.Where do earthquakes occur very frequently?
frequent_locations = location_counts.orderBy(col("count").desc())
frequent_locations.show()

In [ ]:
#9.What is the relation between Magnitude, Magnitude Type, Status, and Root Mean Square of the earthquakes?
magnitude_relation = df_mysql.select("Magnitude", "MagType", "Status", "RootMeanSquare")
magnitude_relation.show()